In [1]:
import tensorflow as tf
STRATEGY = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
NUM_DEVICES = STRATEGY.num_replicas_in_sync

from notebook_utils import create_model, calculate_flops, extract_dataset, extract_ucm, compute_connections, compute_neurons
import nengo
import nengo_dl
import numpy as np
from time import time
from datetime import timedelta

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


## 1) - Creating a model that can be transferred to a spiking model

Spiking models hyperparameters.

In [2]:
scale=0.1 #Scale value
synapse=0.01#Synapse
timesteps=10 #Number of timesteps

Dataset hyperparameters.

In [3]:
dataset_used="UCM" # Accepted: UCM, EuroSAT
dataset_dict = {"UCM" : "../dataset/UCM", "EuroSAT" : "../dataset/EuroSAT"}
img_dict = {"UCM" : (256, 256,3), "EuroSAT" : (64,64,3)}
num_classes_dict = {"UCM" : 21, "EuroSAT" : 10}

In [4]:
dataset_path=dataset_dict[dataset_used]
train_percentage=0.70
test_percentage=0.2
img_shape=img_dict[dataset_used]
n_test=1# Number of images to use (Use -1 to use all.)

## 2) - Creating a model that can be transferred to a spiking model

Model extraction.

In [5]:
model_name = "vgg16" #Model architecture (supported: vgg16 / alexnet)
weights_path= "C:/Users/meoni/Documents/ESA/ACT/SNN/nengo_space_git/Nengo_Space/notebook/models/model.h5"
model, input_layer, output_layer, global_pool = create_model(STRATEGY, model_name=model_name, input_shape=img_shape, num_classes=num_classes_dict[dataset_used], weights_path=weights_path)

Loading weights...


## 3) - Estimating the model flops 

## 4) - Converting model to nengo_dl model

Converting the model.

In [6]:
# Convert to a Nengo network
converter = nengo_dl.Converter(model,
                               scale_firing_rates=scale,
                               synapse=synapse,
                               swap_activations={tf.nn.relu: nengo.LIF()})

network_input = converter.inputs[input_layer]
network_output = converter.outputs[output_layer]

C:\Users\meoni\anaconda3\envs\nengo_space\lib\site-packages\nengo_dl\converter.py:309: UserWarning: block1_conv1.kernel_regularizer has value <tensorflow.python.keras.regularizers.L2 object at 0x000001E331357240> != None, which is not supported (unless inference_only=True). Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
C:\Users\meoni\anaconda3\envs\nengo_space\lib\site-packages\nengo_dl\converter.py:309: UserWarning: block1_conv2.kernel_regularizer has value <tensorflow.python.keras.regularizers.L2 object at 0x000001E331342588> != None, which is not supported (unless inference_only=True). Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
C:\Users\meoni\anaconda3\envs\nengo_space\lib\site-packages\nengo_dl\converter.py:309: UserWarning: block2_conv1.kernel_regularizer has value <tensorflow.python.keras.regularizers.L2 object at 0x000001E331284898> != None, which is not supported (unless inference_only=True). Falling back to TensorNode.
  %

TypeError: Tensors are unhashable. (KerasTensor(type_spec=TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'"))Instead, use tensor.ref() as the key.

Adding probes to the model.

In [ ]:
n_layer = len(converter.layers)#Number of layers
probe_layer = []
with converter.net:
    for layer in model.layers:
        probe_layer.append(nengo.Probe(converter.layers[layer]))
    nengo_dl.configure_settings(stateful=False)

Loading dataset

In [ ]:
if dataset_used == "UCM":
    import tensorflow_datasets as tfds
    [_, _, _, _, test_data, test_label] = extract_ucm(dataset_path, [img_shape[0],img_shape[1]], train_percentage, 1-(train_percentage+test_percentage))
    test_zeros =  np.zeros([len(test_data), test_data[0].shape[0], test_data[1].shape[1], test_data[2].shape[2]])

    for n in range(len(test_data)):
        test_zeros[n] = test_data[n]

    test_ds = tf.data.Dataset.from_tensor_slices((test_zeros, test_label))
    
else:
    [_, _, test_ds] = extract_dataset(dataset_path=dataset_path, train_percentage=train_percentage, test_percentage=test_percentage, batch_size=1, img_size=(img_shape[0], img_shape[1]))

Reducing number of test samples.

In [ ]:
if n_test > -1:
    test_ds = test_ds.take(n_test)

Extracting samples and shaping according to the number of timesteps.

In [ ]:
test_data = [(n[0].numpy(), n[1].numpy()) for n in test_ds.take(n_test)]
x_test = np.array([n[0] for n in test_data])
y_test = np.array([n[1] for n in test_data])

# Tile images according to the number of timesteps
tiled_test_images = np.tile(np.reshape(x_test, (x_test.shape[0], 1, -1)), (1, timesteps, 1))
test_labels = y_test.reshape((y_test.shape[0], 1, -1))

In [ ]:
with nengo_dl.Simulator(converter.net) as sim:
    # Record how much time it takes
    start = time()
    data = sim.predict({network_input: tiled_test_images})
    print('Time to make a prediction with {} timestep(s): {}.'.format(timesteps, timedelta(seconds=time() - start)))

In [ ]:
def energy_estimate_layer(model, layer_idx, spikes_measurements=None, probe_layers=None, dt=0.001, n_timesteps=100, spiking_model=True, device="cpu", if_neurons=False, verbose=False):
    devices = {
        # https://ieeexplore.ieee.org/abstract/document/7054508
        # TODO: CPU neuron energy depends on neuron type
        "cpu": dict(spiking=False, energy_per_synop=8.6e-9, energy_per_neuron=8.6e-9),
        "gpu": dict(spiking=False, energy_per_synop=0.3e-9, energy_per_neuron=0.3e-9),
        "arm": dict(spiking=False, energy_per_synop=0.9e-9, energy_per_neuron=0.9e-9),
        # https://www.researchgate.net/publication/322548911_Loihi_A_Neuromorphic_Manycore_Processor_with_On-Chip_Learning
        "loihi": dict(
            spiking=True,
            energy_per_synop=(23.6 + 3.5) * 1e-12,
            energy_per_neuron=81e-12,
        ),
        # https://arxiv.org/abs/1903.08941
        "spinnaker": dict(
            spiking=True, energy_per_synop=13.3e-9, energy_per_neuron=26e-9
        ),
        "spinnaker2": dict(
            spiking=True, energy_per_synop=450e-12, energy_per_neuron=2.19e-9
        ),
    }
    energy_dict = devices[device]
    if (spiking_model == True) and (energy_dict["spiking"] == False):
        print("Error! Impossible to infer Spiking models on standard hardware!")
        return -1
    
    energy_per_synop = energy_dict["energy_per_synop"]
    energy_per_neuron = energy_dict["energy_per_neuron"]
    n_neurons = compute_neurons(model.layers[layer_idx]) 
    n_connections_per_neuron = compute_connections(model.layers[layer_idx])
    
    if (spiking_model == False):
        f_in = 1/dt
        n_timesteps = 1
        f_out = 1 / dt
    else:
        spikes_out = spikes_measurements[probe_layers[layer_idx]] 
        if layer_idx == 0:
            f_in = 0
        else:
            spikes_in = spikes_measurements[probe_layers[layer_idx - 1]]
            f_in = np.sum(spikes_in)/ (spikes_in.shape[1] * spikes_in.shape[2] * dt)
        
        f_out = np.sum(spikes_out)/ (spikes_out.shape[1] * spikes_out.shape[2] * dt)
    
    print(n_timesteps)
    #synop_energy/inference = energy/op * ops/event * events/s * s/timestep * timesteps/inference
    synop_energy = energy_per_synop * n_connections_per_neuron * n_neurons * f_in * dt * n_timesteps
    if if_neurons: #neurons are update only when an output spike is produced
        #neuron_energy/inference = energy/op * ops/event * events/s * s/timestep * timesteps/inference
        neuron_energy = energy_per_neuron * n_neurons * f_out * dt * n_timesteps
    else: #neurons are update every timestep (for instance, to implement alpha functions)
        #neuron_energy/inference = energy/op * ops/timestep * timesteps/inference
        neuron_energy = energy_per_neuron * n_neurons * n_timesteps
    if verbose:
        print("--- Layer: ", model.layers[layer_idx].name, " ---")
        print("\tSynop energy: ", synop_energy, "J/inference")
        print("\tNeuron energy: ", neuron_energy, "J/inference")
        print("\tTotal energy: ", neuron_energy+synop_energy, "J/inference")
    return neuron_energy,synop_energy

In [ ]:
print("--------- ANN model ---------")
neuron_energy,synop_energy = 0,0
for layer_idx in range(len(model.layers)):
    neuron_energy_layer,synop_energy_layer=energy_estimate_layer(model, layer_idx, spikes_measurements=data, probe_layers=probe_layer, dt=sim.dt, n_timesteps=timesteps, spiking_model=False, device="loihi", verbose=False)
    neuron_energy+=neuron_energy_layer
    synop_energy+=synop_energy_layer
print("--------- Total energy ---------")
print("Synop energy: ", synop_energy, "J/inference")
print("Neuron energy: ", neuron_energy, "J/inference")
print("Total energy: ", synop_energy+neuron_energy," J/inference")

print("--------- SNN model ---------")
neuron_energy_layer,synop_energy_layer = 0,0
for layer_idx in range(len(model.layers)):
    neuron_energy_layer,synop_energy_layer=energy_estimate_layer(model, layer_idx, spikes_measurements=data, probe_layers=probe_layer, dt=sim.dt, n_timesteps=10, spiking_model=True, device="loihi", verbose=False)
    neuron_energy+=neuron_energy_layer
    synop_energy+=synop_energy_layer
print("--------- Total energy ---------")
print("Synop energy: ", synop_energy, "J/inference")
print("Neuron energy: ", neuron_energy, "J/inference")
print("Total energy: ", synop_energy+neuron_energy," J/inference")
